In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = True
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import sys, os
sys.path.append('./pyFM/')
sys.path.append('./VisualizationTools/')


In [ ]:
import large_mesh as lmu
import VisualizationTools as plu

from pyFM.mesh import TriMesh

In [ ]:
mesh1 = TriMesh("./data/40.off",area_normalize=True, center=True).process(k=0, intrinsic=True)
mesh1.n_vertices

In [ ]:
process_params = {
    'dist_ratio': 3,
    'self_limit': .25,
    'correct_dist': False,
    'interpolation': 'poly',
    'return_dist': True,
    'adapt_radius': True,
    'n_jobs':10,
    'n_clusters': 100,
    'verbose': True
}

n_samples = 2000

In [ ]:
U1, Ab1, Wb1, sub1, distmat1 = lmu.process_mesh(mesh1, n_samples, **process_params)
evals1, evects1 = lmu.get_approx_spectrum(Wb1, Ab1, verbose=True)

In [ ]:
plu.plot(mesh1, points=sub1, cmap_p=evects1[:,1], colormap="coolwarm")